In [1]:
from source.util import utils as u
from source import metrics, plots
from source import handshake, handshake2, scargc, hs
import sys
import time
import os
import psutil
import resource
import numpy as np
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [2]:
def doTrain(data, label, step, n_features, percent, epsilon, n_components, k):

    label_ = label

    size = len(data)
    mod = size % 10

    if mod != 0:
        data = data[0:(size - mod), :]
        label = label[0:(size - mod)]

    tam = int((size - mod)/10)
    # print(tam)
    # sys.exit(0)

    data = np.vsplit(data, 10)
    label = u.chunkify(label, 10)
    label_ = label_[0:(size-mod)]
    # print(len(data))
    # print(len(label))

    # percent = int(percent)
    # print(percent)
    # print(epsilon)
    acc_percent = []
    f1_percent = []
    mcc_percent = []
    predicted = []
    updt = []
    
    pool_size = int( len(data[0])/100 * percent )
    score = 0.0
    f1 = 0.0
    mcc = 0.0
    
    if pool_size >= k:
        pred, updt, gmm = hs.handshake2(data, label_, data[0], label[0], data[1], label[1], n_components, n_features, epsilon, percent, k)
        pred = np.array(pred, dtype=int)
        # print(label[i])
        pred = pred.flatten()
        predicted = pred
        acc_percent, f1_percent, mcc_percent = metrics.makeBatches(label[0], predicted, len(label[0]), step)

        # print(acc_percent, f1_percent, mcc_percent)
        # sys.exit(0)

        for i in range(1, len(data)-1):
            # print(i)
            pred, upd, gmm = hs.handshake2(data, label_, data[i], label[i], data[i+1], label[i+1], n_components, n_features, epsilon, percent, k)
            pred = np.array(pred, dtype=int)
            # print(label[i])
            pred = pred.flatten()

            # print(pred)
            acc, f1, mcc = metrics.makeBatches(label[i], pred, len(label[i]), step)

            predicted = np.vstack([predicted, pred])
            # print(predicted)
            # sys.exit(0)

            updt += upd
            acc_percent = np.vstack([acc_percent, acc])
            f1_percent = np.vstack([f1_percent, f1])
            mcc_percent = np.vstack([mcc_percent, mcc])


        # print(acc_percent)

        predicted = predicted.flatten()
        lab = label_[tam:]


        score = accuracy_score(lab, predicted)
        f1 = f1_score(lab, predicted, average = 'macro')
        mcc = matthews_corrcoef(lab, predicted)
        # score, f1, mcc = metrics.metrics(acc_percent, lab, predicted, step, f1_type = 'macro')
        # print(a,score, f1, mcc)
    return score, f1, mcc


In [3]:
poolsize = 300
clusters = 2
n_components = 2
# episilon = 0.15
step = 10

In [4]:
base = '/home/god/Documentos/tcc/datasets/'
list = ['1CSurr.txt', '2CDT.txt', '2CHT.txt']# 'NOAA.txt', 'elec.txt', 'keystroke.txt']
# list = ['keystroke.txt']
# list = ['NOAA.txt']#, 'elec.txt']
database = {}

for i in range(0, len(list)):
    database[i] = base + list[i]

array_ep = [0.05, 0.10, 0.15]
# array_ep = [0.1]
array_p = [10, 20, 30]
# array_p = [30]
array_k = [50, 25, 12, 7, 5, 3, 1]

result = [0.0, 0.0, 0.0]
aux = []
result = np.array(result, dtype=float)

In [5]:
dic = {}
for key, value in database.items():
    dic[int(key)+2] = {}
    for ep in range(0, len(array_ep)):
        for p in range(0, len(array_p)):
            for k in range(0, len(array_k)):
                adr = value
                dataset, data_labeled, dataset_train, l_train, stream, l_stream, n_features = u.criar_datasets(5, adr)

                score, f1, mcc = doTrain(dataset_train, l_train, step, n_features, array_p[p], array_ep[ep], n_components, array_k[k])

                aux = np.hstack([score, f1, mcc])
                result = np.vstack([result, aux])

                a = str(array_ep[ep])
                b = str(array_p[p])
                c = str(array_k[k])
                name = a +' / '+ b + ' / ' + c
                dic[int(key)+2][name] = aux

/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [6]:
for p_id, p_info in dic.items():
    print('dataset :', p_id)

    for key in p_info:
        print(key, ':', p_info[key])

dataset : 2
0.05 / 10 / 50 : [0. 0. 0.]
0.05 / 10 / 25 : [ 0.31884058  0.28762194 -0.41891961]
0.05 / 10 / 12 : [0.55072464 0.54294872 0.12321127]
0.05 / 10 / 7 : [0.5205314  0.50635131 0.03400127]
0.05 / 10 / 5 : [0.57407407 0.56214534 0.14957629]
0.05 / 10 / 3 : [ 0.39573269  0.37437082 -0.23903149]
0.05 / 10 / 1 : [0.55152979 0.52858626 0.06634615]
0.05 / 20 / 50 : [0.53784219 0.53438638 0.12612539]
0.05 / 20 / 25 : [ 0.48268921  0.46968359 -0.03721532]
0.05 / 20 / 12 : [0.55394525 0.54679534 0.1332304 ]
0.05 / 20 / 7 : [0.52133655 0.50759241 0.03727175]
0.05 / 20 / 5 : [0.54991948 0.54546151 0.14280869]
0.05 / 20 / 3 : [ 0.46336554  0.44958806 -0.07893046]
0.05 / 20 / 1 : [ 0.40297907  0.38187337 -0.22375724]
0.05 / 30 / 50 : [0.66143317 0.66010975 0.40761572]
0.05 / 30 / 25 : [0.68276973 0.68271707 0.498458  ]
0.05 / 30 / 12 : [0.63043478 0.62951154 0.44826457]
0.05 / 30 / 7 : [0.66304348 0.6628782  0.49118482]
0.05 / 30 / 5 : [0.61553945 0.61362173 0.4380454 ]
0.05 / 30 / 3 : [0.